In [1]:
%load_ext autoreload
%autoreload 2

%cd ../..
!hostname

/p/fastdata/pli/Private/oberstrass1/datasets/vervet1818-3d
jrlogin09.jureca


In [16]:
pyramid = 6

wm_path = f"data/aa/volume/wm/wm_{pyramid}_smooth.h5"
pial_path = f"data/aa/volume/pial/pial_{pyramid}.h5"
volume_group = "volume"
affine_name = 'affine'

###

import h5py as h5
import numpy as np

with h5.File(wm_path, 'r') as f:
    wm_volume = f[volume_group][:]
    wm_affine = f[affine_name][:]

with h5.File(pial_path, 'r') as f:
    pial_volume = f[volume_group][:]
    pial_affine = f[affine_name][:]

assert np.all(wm_affine == pial_affine)
affine = np.vstack([pial_affine, np.array([[0, 0, 0, 1]])])

print(pial_volume.shape)
print("Affine\n", affine)

(448, 243, 485)
Affine
 [[0.0843776 0.        0.        0.       ]
 [0.        0.06      0.        0.       ]
 [0.        0.        0.0843776 0.       ]
 [0.        0.        0.        1.       ]]


In [17]:
out_file = f"data/aa/volume/brain_visa/cortex_{pyramid}.nii"

BG = 0
GM = 100
WM = 200

###

from pli.data.functions import write_nifti

out_volume = np.full_like(pial_volume, BG, dtype=np.int16)
out_volume[pial_volume] = GM
out_volume[wm_volume] = WM

write_nifti(out_file, out_volume, affine)